In [37]:
import numpy as np
import matplotlib.pyplot as plt
from pymerlin.recon import linear_phyllotaxis, piccini_phyllotaxis
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
from matplotlib import cm
from helpers import *
%matplotlib inline

In [58]:
# Golden ratio
g = (1 + np.sqrt(5))/2

# Golden angle
ag = 2*np.pi/g**2

print("Golden ratio (g): {:.6f}\nGolden angle (ag): {:.2f} deg".format(g, np.rad2deg(ag)))

Golden ratio (g): 1.618034
Golden angle (ag): 137.51 deg


## Introduction
This notebooks gives some additional information about the phyllotaxis trajectory to accompany the paper. There is a function in pymerlin for producing the phyllotaxis trajectory. 

Below we plot the trajectory with different smoothing factors to show how that changes the trajectory. It is plotted as points to better illustrate how the spacing between points changes. **(SI Figure 2)**

In [53]:
fig = plt.figure(figsize=(12,4))
for (i,sf) in enumerate([8,9,10]):
    traj = linear_phyllotaxis(n=256, nint=1, sf=sf)
    ax = fig.add_subplot(1,3,i+1,projection='3d')
    ax.plot(traj[:,0], traj[:,1], traj[:,2], '.k')
    ax.set_xticks([-1,0,1])
    ax.set_yticks([-1,0,1])
    ax.set_zticks([-1,0,1])
    ax.set_xlabel(r'$k_x$')
    ax.set_ylabel(r'$k_y$')
    ax.set_zlabel(r'$k_z$')
    ax.set_title(r'$s={}$'.format(sf), fontsize=14)

plt.savefig("SI_Figure2.png",dpi=300)
plt.show()

### Subsampling for smaller angular steps
The azimuthal angle is incremented by the golden angle for each spoke. Below we demonstrate how subsampling with a factor $x\in F_n$ (where $F_n$ is the Fibonacci sequence) results in smaller and smaller angular steps. **(Figure 1)**

In [57]:
npoints = 1000
n = np.arange(npoints)

# Increment azimuthal angle by golden angle
phi = n*ag
# Produce a linear z gradient
theta = np.arccos(n/npoints*2-1)

# Convert to spherical coordinates
x = np.cos(phi)*np.sin(theta)
y = np.sin(phi)*np.sin(theta)

fig = plt.figure(figsize=(12,3))
labs = ['A', 'B']
for (i,ss) in enumerate([13, 15]):
    fig.add_subplot(1,3,i+1)
    plt.plot(n,x, '.', markersize=3, color='gray')
    plt.plot(n[0::ss],x[0::ss], '-', color='C03', linewidth=2)
    plt.grid()
    plt.axis([0,npoints,-1,1])
    plt.xlabel('Spoke number', fontsize=14)
    plt.ylabel(r'G$_x$', fontsize=14)
    plt.title('({}) Subsampling: {:d}'.format(labs[i], ss), fontsize=16)
    
fig.add_subplot(1,3,3)
phi1 = np.mod(phi,2*np.pi)
mask = phi1>np.pi
phi1[mask] = -1*phi1[mask]+2*np.pi
phi1 *= 360/(2*np.pi)

# Predicted difference
pred_diff = np.rad2deg(2*np.pi/g**(np.arange(2,11)))

fibnum = np.array([fibonacci(x) for x in range(2,11)])
plt.plot(n,phi1, label=r'$x\cdot \alpha_G$', color='C00', linewidth=2)
plt.plot(fibnum, phi1[fibnum], '-', label=r'$x\in F_n$', color='C01', linewidth=2)
plt.plot(fibnum, pred_diff, 'ok', label=r'$\Delta \phi_n$')

plt.xlabel("Spoke number", fontsize=14)
plt.ylabel("Angle [deg]", fontsize=14)
plt.legend(framealpha=1, loc='upper right', fontsize=12, bbox_to_anchor=(1.55,1))
plt.axis([0,60,0,180])
plt.xticks([0,3,5,8,13,21,34,55])
plt.grid()
plt.title('(C) Spoke angle', fontsize=16)
    
plt.tight_layout()
plt.savefig('figure1.png', dpi=300)

In [56]:
print("Spoke differences\n")
print("s \tn \tdphi (deg)")
for x in np.arange(2,11):
    dphi = np.rad2deg(2*np.pi/g**x)
    print("{} \t{} \t{:.2f}".format(x, fibonacci(x), dphi))

Spoke differences

s 	n 	dphi (deg)
2 	1 	137.51
3 	2 	84.98
4 	3 	52.52
5 	5 	32.46
6 	8 	20.06
7 	13 	12.40
8 	21 	7.66
9 	34 	4.74
10 	55 	2.93


### Uniform sampling density
In this work, we wanted to achieve an isotropic field of view, for which the sampling density should be even throughout k-space. The formulation by Piccini et al results in lower sampling density around the poles in k-space, thus an non-isotropic field of view. This is illustrated through the figures below (Note, calculation of the voronoi diagram in 3D takes a while so be patient...) **(SI Figure 1)**

In [52]:
nint = 13
spi = 256
n = nint*spi
vmin=0.8
vmax=1.2
cmap='viridis'

fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,2,1, projection='3d')
traj_pic = piccini_phyllotaxis(n,nint=nint)
voronoi_3D(traj=traj_pic, vmin=vmin, vmax=vmax, cmap=cmap, ax=ax, title='(A) 3D Spiral Phyllotaxis Piccini et al.')

ax = fig.add_subplot(1,2,2, projection='3d')
traj_merlin = linear_phyllotaxis(n, nint, 7)
voronoi_3D(traj=traj_merlin, vmin=vmin, vmax=vmax, cmap=cmap, ax=ax, title='(B) 3D Spiral Phyllotaxis MERLIN')

plt.tight_layout()
plt.savefig('SI_Figure1.png', dpi=300)
plt.show()